# Fantasy Premier League Moneyball

In [ ]:
# The plan:

# link to google sheet:
# https://docs.google.com/spreadsheets/d/1z-DRmtGm3pW-xp951SmK0VTQidB_CxWMNmmciaQQc8M/edit#gid=581306190

# 1. Pull data from fantasy premier league site to create datasets for:
    # Club data - DONE
    # Player data - DONE
    # Fixture data - DONE - add in the fixture multipliers here
    # My team data - DONE - need to add player score once calculated
    # Previous season player data - NEED TO CREATE DF FROM CSV :
        #'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\FPL_Players-2019-2020 season'
    
# 2. Create player score based on:
    # A - Points per million this season - DONE
    # B - Points per million last season (proportionate to games played by team so far this season, with a lower weighting)
    #   - Requires appearances to be calculated 

    # C - Next gameweek multiplier (Includes Matches per gameweek multiplier)
    # D - Next 5 gameweek average multiplier
    
    # E - Chances of playing next round multiplier (excludes injury & suspended) - DONE 
    
    # Player score = (AVERAGE(A x 1.25), (B x 0.75)) x AVERAGE(C, D)) x E  
    
# 3. Create Optimal team:    
    # Create best team of: 
        # Using 80-82 mil
        # 1 x GK
        # 3 x DEF
        # 4 x MID
        # 3 X FOR

    # Leaving enough for Subs: 
        # 18-20mil
        # 1 x GK
        # 2 x DEF
        # 1 X MID

# 4. Figure out a way to make transfer most efficiently to maximise my team score, 
    # Considering 1 free transfer, then 4 points per transfer after that       
    # calculate points predicted to score in next gameweek of lowest in active team 
    # vs 
    # position vs highest points predicted of player not in team
    
# Additional ideas
    # Pull data from promoted teams from the EFL championship in previous year so there is data on newly promoted teams/players
    # Add logic of points per appearance to get around lower points for injured or booked players
    # Possibly mins per appearance to infer a likliness to play
    # Add in a 'strongest fixture of the season' flag for players for when to play triple captain
    # Add in 'strongest fixture of next gameweek' flag for current capatin / vice captain
    

### Creating variables for the api urls and logging in using personal details

In [253]:
#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np


###########################
## URLs an Login Details ##
###########################

# Code for my team picks
my_team_id = '3366741'

# Current Datetime for csv outputs
datetime = dt.now().strftime("%Y%m%d-%H%M%S")

## URLs

## login url to create a session using my login details ## 
login_url = 'https://users.premierleague.com/accounts/login/'

## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/3366741/'

## api for god knows what??
entry_url = 'https://fantasy.premierleague.com/api/entry/3366741/'

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'

session = requests.session()

## Create session with login details for urls which require login ##
payload = {
 'password': 'Football!159',
 'login': 'andrew_c_morris@hotmail.co.uk',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(login_url, data=payload)


<Response [200]>

### Creating data frames and csv outputs

In [289]:
## CLUB DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for the club data and saving as csv ###

############################
#### dataframe creation ####
############################

## Converting the data from the url to json then pulling out the teams data to a separate 'club' data frame
main_req = requests.get(main_url)
main_json = main_req.json()
main_json.keys()
club_df = pd.DataFrame(main_json['teams'])

# print(club_df)

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
club_filename = 'FPL_Clubs-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
club_path = 'C:\\Python CSV Output\\'+club_filename+datetime+'.csv'

## Saved dataframe as csv
club_df.to_csv(club_path, index = False)

In [255]:
### PLAYER DATA EXTRACT ###
### Creating a dataframe for the player data and saving as csv ###

############################
#### Dataframe Creation ####
############################

full_player_df = pd.DataFrame(main_json['elements'])         # Player dataframe
position_types_df = pd.DataFrame(main_json['element_types']) # Position dataframe
events_df = pd.DataFrame(main_json['events'])                # Gameweek dataframe

# print(club_df.head())

# Updating and creating some of the fields in full player dataframe: 
    # Pulling through position and team name instead of 'id', and changing data types
full_player_df['position'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name)
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team'] = full_player_df.team.map(club_df.set_index('id').name)
full_player_df['team_abv'] = full_player_df.team.map(club_df.set_index('name').short_name)

    # Converting selected_by_percent to float data type
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

    # Creating chance of playing next round multiplier
full_player_df['next_round_play_chance'] = full_player_df['chance_of_playing_next_round']/100.0
full_player_df['next_round_play_chance'] = full_player_df.next_round_play_chance.fillna(1.0)
full_player_df['this_round_play_chance'] = full_player_df['chance_of_playing_this_round']/100.0
full_player_df['this_round_play_chance'] = full_player_df.this_round_play_chance.fillna(1.0)

    # Creating custom point_per_mil
full_player_df['cost_in_mil'] = full_player_df['now_cost']/10
full_player_df['point_per_mil'] = round(full_player_df['total_points']/full_player_df['cost_in_mil'],2)

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name',
                                 'second_name',
                                 'web_name',   
                                 'id',   
                                 'position',
                                 'position_abv',
                                 'team',
                                 'team_abv',
                                 'selected_by_percent', 
                                 'this_round_play_chance',   
                                 'next_round_play_chance',      
                                 'status',
                                 'cost_in_mil',
                                 'minutes',
                                 'transfers_in',
                                 'transfers_out',   
                                 'total_points',
                                 'point_per_mil']]


print(slim_player_df)

#########################################
# print(full_player_df)
# print(slim_player_df)
# print(slim_player_has_played_df)
# print(average_cost_df)
#########################################

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
player_filename = 'FPL_Players-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
player_path = 'C:\\Python CSV Output\\'+player_filename+datetime+'.csv'

# ## Exporting dataframe as csv
# slim_player_df.to_csv(player_path, index = False, encoding="utf8")

# ## Other tables to export
# full_player_df.to_csv(player_path, index = False, encoding="utf8")
# slim_player_has_played_df.to_csv(player_path, index = False, encoding="utf8")

         first_name           second_name     web_name   id    position  \
0             Mesut                  Özil         Özil    1  Midfielder   
1          Sokratis      Papastathopoulos     Sokratis    2    Defender   
2             David  Luiz Moreira Marinho   David Luiz    3    Defender   
3    Pierre-Emerick            Aubameyang   Aubameyang    4  Midfielder   
4            Cédric                Soares       Cédric    5    Defender   
..              ...                   ...          ...  ...         ...   
522     Bruno André         Cavaco Jordao       Jordao  475  Midfielder   
523        Leonardo               Campana      Campana  476     Forward   
524           Fabio                 Silva  Fabio Silva  504     Forward   
525        Fernando                Marçal       Marçal  505    Defender   
526           Vitor              Ferreira      Vitinha  515  Midfielder   

    position_abv     team team_abv  selected_by_percent  \
0            MID  Arsenal      ARS      

In [292]:
### FIXTURE DATA EXTRACT ###
### Creating a dataframe for the fixtures data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

# Adding club names and abbreviations as new columns
fix_df['home_team_full']  = fix_df.team_h.map(club_df.set_index('id').name)
fix_df['home_team_short'] = fix_df.team_h.map(club_df.set_index('id').short_name)
fix_df['away_team_full']  = fix_df.team_a.map(club_df.set_index('id').name)
fix_df['away_team_short'] = fix_df.team_a.map(club_df.set_index('id').short_name)

# Adding Home/Away Strengths to fix_df - 
# The away and home strengths seem to be transposed for nearly all teams (teams have higher away strength than home strength)
# So I have switched them around here
# Keep an eye on this to see if they flip around at any point
    # Home team strength scores
fix_df['h_team_str_overall_home'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_away)
fix_df['h_team_str_att_home'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_away)
fix_df['h_team_str_def_home'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_away)
fix_df['h_team_str_overall_away'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_home)
fix_df['h_team_str_att_away'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_home)
fix_df['h_team_str_def_away'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_home)

    # Away team strength scores
fix_df['a_team_str_overall_away'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_home)
fix_df['a_team_str_att_away'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_home)
fix_df['a_team_str_def_away'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_home)
fix_df['a_team_str_overall_home'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_away)
fix_df['a_team_str_att_home'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_away)
fix_df['a_team_str_def_home'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_away)

# Creating home/away position specific multipliers
    # Basic logic is:
        # For goalkeepers or defenders - home team defense score / away team attack score
        # For midfielders - home team overall score / away team overall score
        # For forwards - home team attack score / away team defense score
    
    # Home team multipliers
fix_df['home_GKP_DEF_multi'] = fix_df['h_team_str_def_home'] / fix_df['a_team_str_att_away']
fix_df['home_MID_multi'] = fix_df['h_team_str_overall_home'] / fix_df['a_team_str_overall_away']
fix_df['home_FWD_multi'] = fix_df['h_team_str_att_home'] / fix_df['a_team_str_def_away']

    # Away team multiplier
fix_df['away_GKP_DEF_multi'] = fix_df['a_team_str_def_away'] / fix_df['h_team_str_att_home']
fix_df['away_MID_multi'] = fix_df['a_team_str_overall_away'] / fix_df['h_team_str_overall_home']
fix_df['away_FWD_multi'] = fix_df['a_team_str_att_away'] / fix_df['h_team_str_def_home']

# Renaming 'event' to 'gameweek'
fix_df = fix_df.rename(columns={"event": "gameweek"})

# print(fix_df)

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
full_fixtures_filename = 'FPL_Full_Fixtures-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
fixture_path = 'C:\\Python CSV Output\\'+full_fixtures_filename+datetime+'.csv'

## Exporting dataframe as csv
# fix_df.to_csv(fixture_path, index = False)

In [314]:
#######################################################
### SLIM FIXTURES DATAFRAME AND GAMEWEEK MULTIPLIER ###
#######################################################

# Creating a slim version of the fixtures dataframe  

home_fix_df = fix_df[['gameweek',
                      'kickoff_time',
                      'team_h',
                      'home_team_full',
                      'home_team_short',
                      'away_team_full',
                      'away_team_short',
                      'team_h_difficulty',
                      'home_GKP_DEF_multi',
                      'home_MID_multi',
                      'home_FWD_multi'
                     ]]

home_fix_df['home_or_away'] = 'home'

home_fix_df = home_fix_df.rename(columns={'team_h':'team_id',
                                          'home_team_full':'team',
                                          'home_team_short':'team_abv',
                                          'away_team_full':'opponent',
                                          'away_team_short':'opponent_abv',
                                          'team_h_difficulty':'match difficulty',
                                          'home_GKP_DEF_multi':'GKP_DEF_multi',
                                          'home_MID_multi':'MID_multi',
                                          'home_FWD_multi':'FWD_multi'
                                         })


away_fix_df = fix_df[['gameweek',
                      'kickoff_time',
                      'team_a',
                      'away_team_full',
                      'away_team_short',
                      'home_team_full',
                      'home_team_short',
                      'team_a_difficulty',
                      'away_GKP_DEF_multi',
                      'away_MID_multi',
                      'away_FWD_multi'
                     ]]

away_fix_df['home_or_away'] = 'away'

away_fix_df = away_fix_df.rename(columns={'team_a':'team_id',
                                          'away_team_full':'team',
                                          'away_team_short':'team_abv',
                                          'home_team_full':'opponent',
                                          'home_team_short':'opponent_abv',
                                          'team_a_difficulty':'match difficulty',
                                          'away_GKP_DEF_multi':'GKP_DEF_multi',
                                          'away_MID_multi':'MID_multi',
                                          'away_FWD_multi':'FWD_multi'
                                         })

slim_fix_df = pd.concat([home_fix_df,away_fix_df])
slim_fix_df = new_fix_df.sort_values(by=['kickoff_time']).reset_index()
print(slim_fix_df.head(10))

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
fixtures_filename = 'FPL_Fixtures-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
fixture_path = 'C:\\Python CSV Output\\'+fixtures_filename+datetime+'.csv'

## Exporting dataframe as csv
slim_fix_df.to_csv(fixture_path, index = False)

   level_0  index  gameweek          kickoff_time  team_id            team  \
0        0      2       1.0  2020-09-12T11:30:00Z        8          Fulham   
1        1      2       1.0  2020-09-12T11:30:00Z        1         Arsenal   
2        2      3       1.0  2020-09-12T14:00:00Z        6  Crystal Palace   
3        3      3       1.0  2020-09-12T14:00:00Z       16     Southampton   
4        4      4       1.0  2020-09-12T16:30:00Z       11       Liverpool   
5        5      4       1.0  2020-09-12T16:30:00Z       10           Leeds   
6        6      5       1.0  2020-09-12T19:00:00Z       19        West Ham   
7        7      5       1.0  2020-09-12T19:00:00Z       14       Newcastle   
8        9      6       1.0  2020-09-13T13:00:00Z        9       Leicester   
9        8      6       1.0  2020-09-13T13:00:00Z       18       West Brom   

  team_abv        opponent opponent_abv  match difficulty  GKP_DEF_multi  \
0      FUL         Arsenal          ARS                 3       0

C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\andre\anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [315]:
## MY TEAM PICKS DATAFRAME AND CSV EXTRACT ##
### Creating a dataframe for my team picks data and saving as csv ###

############################
#### dataframe creation ####
############################

## Getting my team data from the api url, using session as login details required
my_team_ses = (session.get('https://fantasy.premierleague.com/api/my-team/3366741'))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()
my_team_picks_df = pd.DataFrame(my_team_json['picks'])

# Pulling through player data from 'full_player_df'
my_team_picks_df['first_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').first_name)
my_team_picks_df['second_name'] = my_team_picks_df.element.map(full_player_df.set_index('id').second_name)
my_team_picks_df['potential_profit'] = my_team_picks_df.purchase_price - my_team_picks_df.purchase_price
my_team_picks_df['total_points'] = my_team_picks_df.element.map(full_player_df.set_index('id').total_points)
my_team_picks_df['point_per_mil'] = my_team_picks_df.element.map(full_player_df.set_index('id').point_per_mil)
my_team_picks_df['position_abv'] = my_team_picks_df.element.map(full_player_df.set_index('id').position_abv)

print(my_team_picks_df)

#####################
##### csv output ####
#####################

## Specifying the filename and path (Location to export the CSV to)
my_team_filename = 'FPL_My_Team-'
datetime = dt.now().strftime("%Y%m%d-%H%M%S")
my_team_path = 'C:\\Python CSV Output\\'+my_team_filename+datetime+'.csv'

## Exporting dataframe as csv
# my_team_picks_df.to_csv(my_team_path, index = False)

    element  position  selling_price  multiplier  purchase_price  is_captain  \
0       217         1             55           1              55       False   
1       259         2             75           1              75       False   
2       353         3             50           1              50       False   
3       457         4             60           1              60       False   
4       478         5             80           1              80       False   
5       244         6             55           1              55       False   
6       355         7             55           1              55       False   
7       254         8            120           2             120        True   
8       460         9             85           1              85       False   
9       366        10             85           1              85       False   
10      224        11            100           1             100       False   
11       70        12             45    

### Random Bits / In Development

In [ ]:
##############################################
####### Show Datatypes Of Dataframe ##########
##############################################

# dataTypeSeries = <insert dataframe name>.dtypes
# print('Data type of each column of Dataframe :')
# print(dataTypeSeries)

In [ ]:
###########################################
### print column names from a dataframe ###
###########################################

# for col in club_df.columns: 
#     print(col) 

In [ ]:
## LOGIC TO MAKE A PIVOT TABLE FOR AVERAGE COST PER POSITION (ONLY PLAYERS WHO HAVE PLAYED) AND CONVERT TO A DATAFRAME ##
# The plan was to blend this with player value (points per mil) 
# but I realised this would be detrimental to those players who are less than the average value
# so I shelved this logic

# There might be something in belnd if player cost more than average else just use the points per mil, 
# so that more expensive players are not penalised as much for their higher price

# Without logic like this the very top expensive players like Salah would not have made it into my 2020 team 
# and therefore not banked 40 points as my captain in Gameweek 1!

# Adding Average Cost of position and points per average mil cost of position:

    # A cut of the slim player table with only players who have played for more than 0 minutes
player_has_played_df = full_player_df[full_player_df.minutes != 0]
    # Creating a pivot table to calculate the average cost for the position (only using players who have played this season)
average_cost_pivot = player_has_played_df.pivot_table(index=['position_abv'],
                                                      values=['cost_in_mil'], 
                                                      aggfunc='mean')
    # Then converting to a dataframe and renaming 
average_cost_df = pd.DataFrame(average_cost_pivot.to_records()).rename(columns={"cost_in_mil": "avg_position_cost"})

In [229]:
##############################################
# Player History - To calculate games played #
##############################################

# Creating an empty list and dictionary
row_list = []
dict1 = {}

# Creating a range of all IDs in 'full_player_df'
id_range = range(1,len(full_player_df)+1)

# For loop to add each individual player json to the row_list 
for i in id_range:
    i_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(i)+'/'
    i_req = requests.get(i_url)
    i_json = i_req.json()
    dict1 = {}
    dict1.update(i_json)
    row_list.append(dict1)

# Converting the row_list into a dataframe   
individual_player_df = pd.DataFrame(row_list)

# Creating a dataframe of just the History part of the above dataframe
# 'Element maps' to id in full_player_df 
player_history_df = pd.DataFrame(individual_player_df['history'])
print(player_history_df)
 
# Need to convert this into a proper dataframe somehow!!!    

                                            fixtures  \
0  [{'id': 9, 'code': 2128296, 'team_h': 1, 'team...   
1  [{'id': 9, 'code': 2128296, 'team_h': 1, 'team...   
2  [{'id': 9, 'code': 2128296, 'team_h': 1, 'team...   
3  [{'id': 9, 'code': 2128296, 'team_h': 1, 'team...   
4  [{'id': 9, 'code': 2128296, 'team_h': 1, 'team...   

                                             history  \
0  [{'element': 1, 'fixture': 2, 'opponent_team':...   
1  [{'element': 2, 'fixture': 2, 'opponent_team':...   
2  [{'element': 3, 'fixture': 2, 'opponent_team':...   
3  [{'element': 4, 'fixture': 2, 'opponent_team':...   
4  [{'element': 5, 'fixture': 2, 'opponent_team':...   

                                        history_past  
0  [{'season_name': '2013/14', 'element_code': 37...  
1  [{'season_name': '2018/19', 'element_code': 39...  
2  [{'season_name': '2010/11', 'element_code': 41...  
3  [{'season_name': '2017/18', 'element_code': 54...  
4  [{'season_name': '2015/16', 'element_code': 58..